In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from netCDF4 import Dataset
import dill

Look at variables in ERA data to figure out latitude/longitude constraints

In [ ]:
nc_file = Dataset('/localhome/data/constants_5.625deg.nc', mode='r')

for variable in nc_file.variables:
    data = nc_file.variables[variable][:]
    print(f"{variable} first 360 values:", data.flat[:360])

for var_name in nc_file.variables:
    var = nc_file.variables[var_name]
    print(var_name, var)

nc_file.close()

Converting ERA5 to be over the contiguous US

In [4]:
existing_mmap = np.memmap('/localhome/data/severestorms/era5625_aaai/era5625_aaai__era5625.mmap',\
     dtype='float32', mode='r+')
existing_mmap_const = np.memmap('/localhome/data/severestorms/era5625_aaai/era5625_aaai__era5625_const.mmap',\
     dtype='float32', mode='r+')

us_mmap = np.memmap('/localhome/data/severestorms/era5625_aaai/era5625_us.mmap',\
     dtype='float32', mode='w+', shape=(359400, 18, 5, 12))
us_mmap_const = np.memmap('/localhome/data/severestorms/era5625_aaai/era5625_us_const.mmap',\
               dtype='float32', mode='w+', shape=(359400, 18, 5, 12)) 

us_mmap[:] = existing_mmap[:,:,20:25,9:21] 
us_mmap_const[:] = existing_mmap_const[:,:,20:25,9:21] 

Create dill files from text files with mmap name + dimensions + each variable

In [ ]:
storm_dill_dict = {'variables': {'imerg5625/precipitationcal': {'name': 'precipitationcal', 'mmap_name': 'storm_data_us.mmap', 'type': 'temp', 'dims': (5, 12), 'offset': 0, 'first_ts': 283996800.0, 'last_ts': 1577833200.0, 'tfreq_s': 3600, 'levels': [0]}}, 'memmap': {'storm_data_us.mmap': {'dims': (359400, 1, 5, 12), 'dtype': 'float32', 'daterange': (283996800.0, 1577833200.0), 'tfreq_s': 3600}}}

dill_file_path = '/localhome/data/rainbench/imerg5625/storm_data_us.dill'

with open(dill_file_path, 'wb') as file:
    dill.dump(storm_dill_dict, file)